In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import BaggingRegressor
from tensorflow import keras
import os
from PIL import Image

import matplotlib.pyplot as plt
import requests
import seaborn as sns
import pickle

import tensorflow as tf
from tensorflow import keras
from keras import metrics


from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error


2023-12-12 15:56:09.147193: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from sklearn.ensemble import BaggingRegressor

def build_bagging_model(base_estimator, n_estimators=10, max_samples=1.0, max_features=1.0, bootstrap=True, bootstrap_features=False, random_state=0):
    """
    Build a bagging regressor model
    :param base_estimator: The base model to ensemble
    :param n_estimators: The number of base estimators in the ensemble
    :param max_samples: The fraction of samples to draw from X to train each base estimator
    :param max_features: The fraction of features to draw from X to train each base estimator
    :param bootstrap: Whether samples are drawn with replacement
    :param bootstrap_features: Whether features are drawn with replacement
    :param random_state: Controls the randomness of the estimator
    :return: A bagging regressor model
    """
    model = BaggingRegressor(
        base_estimator=base_estimator,
        n_estimators=n_estimators,
        max_samples=max_samples,
        max_features=max_features,
        bootstrap=bootstrap,
        bootstrap_features=bootstrap_features,
        random_state=random_state
    )
    return model

# Example usage with a Decision Tree Regressor as the base model
dt_regressor = DecisionTreeRegressor(max_depth=10, random_state=0)
bagging_model = build_bagging_model(dt_regressor)


In [4]:
# import train and test data
x_train = np.load('/Users/natachamaheshe/Documents/GitHub/207_final_project/Data/saved_data/x_train_3500_balanced.npy')
y_train = np.load('/Users/natachamaheshe/Documents/GitHub/207_final_project/Data/saved_data/y_train_3500_balanced.npy')

x_test = np.load('/Users/natachamaheshe/Documents/GitHub/207_final_project/Data/saved_data/x_test_1500_balanced.npy')
y_test = np.load('/Users/natachamaheshe/Documents/GitHub/207_final_project/Data/saved_data/y_test_1500_balanced.npy')


In [5]:
# Check the shape of the datasets
print("Shape of x_train:", x_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of x_test:", x_test.shape)
print("Shape of y_test:", y_test.shape)


Shape of x_train: (3500, 338, 280, 3)
Shape of y_train: (3500, 338, 280)
Shape of x_test: (1500, 338, 280, 3)
Shape of y_test: (1500, 338, 280)


In [6]:
# Flatten x_train and x_test
x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)

# Flatten y_train and y_test
# WARNING: Only do this if it makes sense for your regression task
y_train_flat = y_train.reshape(y_train.shape[0], -1)
y_test_flat = y_test.reshape(y_test.shape[0], -1)

# Check the new shapes
print("New shape of x_train_flat:", x_train_flat.shape)
print("New shape of y_train_flat:", y_train_flat.shape)
print("New shape of x_test_flat:", x_test_flat.shape)
print("New shape of y_test_flat:", y_test_flat.shape)


New shape of x_train_flat: (3500, 283920)
New shape of y_train_flat: (3500, 94640)
New shape of x_test_flat: (1500, 283920)
New shape of y_test_flat: (1500, 94640)


In [7]:
# Building and fitting the model
dt_regressor = DecisionTreeRegressor(max_depth=10, random_state=0)
bagging_model = build_bagging_model(dt_regressor)
bagging_model.fit(x_train_flat, y_train_flat)

/Users/natachamaheshe/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


In [ ]:
# Prediction with the bagging regressor
y_pred_ts_bagging = bagging_model.predict(x_test_flat)

In [28]:
def calculate_mse(y_true, y_pred):
  return np.mean((y_true - y_pred)**2)


In [28]:
def calculate_mae(y_true, y_pred):
  return np.mean(np.abs(y_true - y_pred)**2))


In [28]:
def calculate_r2(y_true, y_pred):
  return 1 - np.sum((y_true - y_pred)**2) / np.sum((y_true - np.mean(y_true))**2)


In [ ]:
# Evaluate the bagging regressor
mse_bagging = calculate_mse(y_pred_ts_bagging, y_test_flat)
rmse_bagging = np.sqrt(mse_bagging)
mae_bagging = calculate_mae(y_pred_ts_bagging, y_test_flat)
r2_bagging = calculate_r2(y_pred_ts_bagging, y_test_flat)

print("Bagging MSE: ", mse_bagging)
print("Bagging RMSE: ", rmse_bagging)
print("Bagging MAE: ", mae_bagging)
print("Bagging R2: ", r2_bagging)


NameError: name 'y_pred_ts_bagging' is not defined